# CVDDivergence Strategy Analysis

This notebook provides an in-depth analysis of the CVDDivergence strategy, including its logic, indicators, and backtest results.

## Setup

### Change Working directory to repository root

In [1]:
from constants import set_cwd_to_project_root, CONFIG_DIR
from main import generate_config

generate_config("binance")

set_cwd_to_project_root()

### Configure Freqtrade environment

In [2]:
from freqtrade.configuration import Configuration

config = Configuration.from_files([CONFIG_DIR])

config.update(
    {
        "timeframe": "5m",
        "strategy": "CVDDivergence"
    }
)

# Pair to analyze - Only use one pair here
PAIR = "BTC/USDT:USDT"

In [3]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from constants import DATA_DIR

candles = load_pair_history(
    datadir=DATA_DIR / "binance",
    timeframe=config["timeframe"],
    pair=PAIR,
    data_format="feather",
    candle_type=CandleType.FUTURES,
)

# Confirm success
print(f"Loaded {len(candles)} rows of data for {PAIR} from {DATA_DIR}")
candles.head()

Loaded 77949 rows of data for BTC/USDT:USDT from F:\quantative-analysis\data


,date,open,high,low,close,volume
0,2024-01-01 00:00:00+00:00,42314.0,42437.2,42289.6,42437.1,1724.210
1,2024-01-01 00:05:00+00:00,42437.2,42474.1,42420.5,42446.8,994.003
2,2024-01-01 00:10:00+00:00,42446.8,42535.0,42445.2,42532.5,899.775
3,2024-01-01 00:15:00+00:00,42532.4,42603.2,42494.1,42494.1,1291.232
4,2024-01-01 00:20:00+00:00,42494.1,42533.1,42484.4,42509.4,418.463


### Load and run strategy

In [4]:
# Load strategy using values set above
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": PAIR})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77949 entries, 0 to 77948
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   date                    77949 non-null  datetime64[ns, UTC]
 1   open                    77949 non-null  float64            
 2   high                    77949 non-null  float64            
 3   low                     77949 non-null  float64            
 4   close                   77949 non-null  float64            
 5   volume                  77949 non-null  float64            
 6   trades                  3000 non-null   object             
 7   orderflow               3000 non-null   object             
 8   imbalances              3000 non-null   object             
 9   stacked_imbalances_bid  611 non-null    object             
 10  stacked_imbalances_ask  519 non-null    object             
 11  max_delta               3000 non-null   f

### Display the trade details

In [5]:
# Report results
print(f"Generated {df['enter_long'].sum()} long entry signals")
print(f"Generated {df['enter_short'].sum()} short entry signals")
data = df.set_index("date", drop=False)

Generated 0.0 long entry signals
Generated 0.0 short entry signals


## Load backtest results

In [6]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
from constants import BACKTEST_RESULTS_DIR

# Load backtest statistics
stats = load_backtest_stats(BACKTEST_RESULTS_DIR)

strategy = "CVDDivergence"
# Example usages:
print("Results per pair:")
print(stats["strategy"][strategy]["results_per_pair"])
print("\nPairlist:")
print(stats["strategy"][strategy]["pairlist"])
print(f"\nMarket change: {stats['strategy'][strategy]['market_change']}")
print(f"Drawdown start: {stats['strategy'][strategy]['drawdown_start']}")
print(f"Drawdown end: {stats['strategy'][strategy]['drawdown_end']}")

# Load backtested trades as dataframe
trades = load_backtest_data(BACKTEST_RESULTS_DIR)

if len(trades) > 0:
    print("\nExit reasons per pair:")
    print(trades.groupby("pair")["exit_reason"].value_counts())

Results per pair:


KeyError: 'CVDDivergence'

## Plot results

In [7]:
from freqtrade.plot.plotting import generate_candlestick_graph
import plotly.graph_objs as go
from pandas import notna

from_date = data["delta"].dropna().index.min()
to_date = data["delta"].dropna().index.max()

# Filter data for a specific date range
data_filtered = data[from_date:to_date].drop(columns=["exit_short", "exit_long"])

# Generate candlestick graph with indicators for trade entry and losses
graph = generate_candlestick_graph(
    pair=PAIR,
    data=data_filtered,
    trades=trades,
    indicators1=["vwap", "vwap_lower_2", "vwap_upper_2", "bull_poc_upper", "bull_poc_lower", "bear_poc_lower", "bear_poc_upper", "pivot"],
    indicators2=["cvd"]
)

# graph.add_trace(
#     go.Scatter(
#         x=data_filtered.index,
#         y=data_filtered['close'],
#         text=data_filtered['pivot'],
#         mode='text',
#         textposition='top center',
#         textfont=dict(size=8),
#         showlegend=False
#     )
# )

# # Adjust layout to prevent overlap
# graph.update_layout(
#     annotations=[
#         dict(
#             x=data_filtered.index[i],
#             y=data_filtered['close'].iloc[i],
#             text=str(data_filtered['pivot'].iloc[i]),
#             showarrow=False,
#             yshift=10,
#             font=dict(size=8)
#         )
#         for i in range(len(data_filtered)) if notna(data_filtered['pivot'].iloc[i])
#     ]
# )

# # Add price_trend label to the graph
# graph.add_trace(
#     go.Scatter(
#         x=data_filtered.index,
#         y=data_filtered['close'],
#         mode='markers+text',
#         text=data_filtered['price_trend'],
#         textposition='top right',
#         textfont=dict(size=8),
#         marker=dict(size=2, color='rgba(0,0,0,0)'),
#         name='Price Trend',
#         showlegend=False
#     )
# )

# Adjust layout to prevent overlap
# graph.update_layout(
#     annotations=[
#         dict(
#             x=data_filtered.index[i],
#             y=data_filtered['close'].iloc[i],
#             text=data_filtered['price_trend'].iloc[i],
#             showarrow=False,
#             yshift=20,
#             xshift=10,
#             font=dict(size=8)
#         )
#         for i in range(len(data_filtered)) if notna(data_filtered['price_trend'].iloc[i])
#     ]
# )



# Render graph in a separate window
graph.show(renderer="browser")

NameError: name 'trades' is not defined

## Plotting daily profit / equity line

In [8]:
import pandas as pd
import plotly.express as px

strategy_stats = stats["strategy"][strategy]

equity = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
equity["equity_daily"] = equity["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily", title="CVDDivergence Strategy - Equity Curve")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['date', 'open', 'high', 'low', 'close', 'volume', 'trades', 'orderflow', 'imbalances', 'stacked_imbalances_bid', 'stacked_imbalances_ask', 'max_delta', 'min_delta', 'bid', 'ask', 'delta', 'total_trades', 'day', 'cvd', 'vwap', 'vwap_upper_1', 'vwap_lower_1', 'vwap_upper_2', 'vwap_lower_2', 'bull_poc_lower', 'bull_poc_upper', 'bear_poc_lower', 'bear_poc_upper', 'atr', 'rsi', 'enter_tag', 'low_sma', 'cvd_sma', 'high_sma', 'enter_long', 'enter_short', 'exit_tag', 'exit_long', 'exit_short'] but received: dates